Importing libraries

In [1]:
import pandas as pd
from google.cloud import vision, storage
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\storage_key.json"

Reading csv file containing product_name and url

In [18]:
data = pd.read_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\data.csv")
data.shape

(100, 2)

Storing url column in data_url

In [19]:
data_url = data['product_url']
data_url

0     https://m.media-amazon.com/images/I/6176qXxgtI...
1     https://m.media-amazon.com/images/I/71XJK6bWJK...
2     https://m.media-amazon.com/images/I/81VUG3Eh5Z...
3     https://m.media-amazon.com/images/I/615wnuU9Zq...
4     https://m.media-amazon.com/images/I/71vE0KfTNQ...
                            ...                        
95    https://m.media-amazon.com/images/I/71BXLqma3B...
96    https://m.media-amazon.com/images/I/41zDXFipUm...
97    https://m.media-amazon.com/images/I/71HxhWJlza...
98    https://m.media-amazon.com/images/I/71NNd7bV8P...
99    https://m.media-amazon.com/images/I/61PIVnN9zT...
Name: product_url, Length: 100, dtype: object

Function to check if url contains a image or not

In [20]:
import requests
def is_url_image(image_url):
   image_formats = ("image/png", "image/jpeg", "image/jpg")
   r = requests.head(image_url)
   if r.headers["content-type"] in image_formats:
      # print(r.headers["content-type"])
      return True
   return False

In [5]:
import webcolors
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

requested_colour = (119, 172, 152)
actual_name, closest_name = get_colour_name(requested_colour)

In [7]:

def detect_properties_uri(uri):
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    # print("Properties:",props.dominant_colors.colors)
    # scr = [col for col in props.dominant_colors.colors]
    # print(scr)
    for color in props.dominant_colors.colors:
        try:
            r = int(color.color.red)
            g = int(color.color.green)
            b = int(color.color.blue)
            print(f"color: {get_colour_name((r,g,b))}")
        except Exception as e:
            print("gone from here",e)
            continue
        
detect_properties_uri(data_url[1])

color: (None, 'black')
color: (None, 'darkslategray')
color: (None, 'rosybrown')
color: (None, 'whitesmoke')
color: (None, 'darkslategray')
color: (None, 'black')
color: (None, 'darkslategray')
color: (None, 'gray')
color: (None, 'black')
color: (None, 'darkslategray')


Empty String variable to store results

In [21]:
txt = ""

Function to extract information from image

In [22]:
def extract_info(uri):
    global txt
    try:
        # if(is_url_image(uri)==True):
            client = vision.ImageAnnotatorClient()
            img = vision.Image()
            img.source.image_uri = uri
            
            #For retriving Brand
            # res_logo = client.logo_detection(img)
            # logos = res_logo.logo_annotations
            # logo = [log.description for log in logos]
            # txt = txt + f"Brand: None, " if len(logo)==0 else txt + f"Brand: {logo[0]}, "
            
            #For retriving Labels
            res_label = client.label_detection(img)
            labels = res_label.label_annotations
            label = [lab.description for lab in labels]
            # txt = txt + f"Description: None, " if len(label)==0 else txt + f"Description: {str(label)}, "
            
            #For retriving Text
            # res_text = client.document_text_detection(img)
            # texts = res_text.text_annotations
            # text = [t.description for t in texts]
            # txt = txt + f"Text: None, " if len(text)==0 else txt + f"Text: {text[0]}, "
            
            objects = client.object_localization(image=img).localized_object_annotations
            obj = [ob.name for ob in objects]
            txt = txt + f"{label}" if len(obj)==0 else txt + f"{obj}"
        
        # else:
        #     txt = "Please enter a valid URL that contains Image"
    except Exception as e:
        txt = f"Please Enter a Valid URL of Image"
        return

Creating new dataframe to save results

In [23]:
result = data
result.head()

,product_name,product_url
0,Hydro Flask Stainless Steel Wide Mouth Water B...,https://m.media-amazon.com/images/I/6176qXxgtI...
1,Owala FreeSip Insulated Stainless Steel Water ...,https://m.media-amazon.com/images/I/71XJK6bWJK...
2,"Gatorade Gx, Marble Blue, 30 Oz",https://m.media-amazon.com/images/I/81VUG3Eh5Z...
3,"Sahara Sailor Water Bottles, 32oz Motivational...",https://m.media-amazon.com/images/I/615wnuU9Zq...
4,"Insulated Water Bottle 64 oz, Triple Wall Vacu...",https://m.media-amazon.com/images/I/71vE0KfTNQ...


Iterating through function to extract features of image and adding it to result dataframe

In [24]:
for i in range(len(data_url)):
    txt = ""
    extract_info(data_url[i])
    result.loc[i, "objects_extracted"] = txt
    

Resultant Dataframe

In [25]:
result

,product_name,product_url,objects_extracted
0,Hydro Flask Stainless Steel Wide Mouth Water B...,https://m.media-amazon.com/images/I/6176qXxgtI...,['Vacuum flask']
1,Owala FreeSip Insulated Stainless Steel Water ...,https://m.media-amazon.com/images/I/71XJK6bWJK...,['Vacuum flask']
2,"Gatorade Gx, Marble Blue, 30 Oz",https://m.media-amazon.com/images/I/81VUG3Eh5Z...,['Bottle']
3,"Sahara Sailor Water Bottles, 32oz Motivational...",https://m.media-amazon.com/images/I/615wnuU9Zq...,"['Bottle', 'Bottle']"
4,"Insulated Water Bottle 64 oz, Triple Wall Vacu...",https://m.media-amazon.com/images/I/71vE0KfTNQ...,"['Vacuum flask', 'Bottle']"
...,...,...,...
95,Baybee R7 Battery Operated Bike for Kids,https://m.media-amazon.com/images/I/71BXLqma3B...,"['Motorcycle', 'Tire', 'Wheel', 'Tire', 'Wheel..."
96,GORMERY 1/12 Scale Model Compatible with Kawasaki,https://m.media-amazon.com/images/I/41zDXFipUm...,"['Motorcycle', 'Tire', 'Tire', 'Wheel', 'Wheel']"
97,Nexus Product Bike for Kids Toy R3 Bike,https://m.media-amazon.com/images/I/71HxhWJlza...,"['Tire', 'Wheel', 'Motorcycle', 'Wheel', 'Tire..."
98,PLUSPOINT Diecast Motorcycle Toy Ducati Bike,https://m.media-amazon.com/images/I/71NNd7bV8P...,"['Motorcycle', 'Wheel', 'Tire', 'Wheel', 'Tire..."


Saving Dataframe to CSV file

In [26]:
result.to_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv",index=False)

Uploading csv file to Cloud Stroage

In [2]:
storage_cli = storage.Client()
bucket = storage_cli.get_bucket("bucket-shreyash")
blob = bucket.blob("Extracted_features_from_image.csv")
blob.upload_from_filename("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")
print("completed")

completed
